<a href="https://colab.research.google.com/github/MKShim/Python/blob/main/RecomSystem_Opt_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 연결

In [ ]:
pip install duckdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 521 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import numpy.matlib
import pandas as pd
import duckdb

#from functools import wraps
#from typing import List, Optional, Tuple, cast
#import time
#import functools
#import pdb, sys

## Data import, preprocessing

In [ ]:
sparse = pd.read_csv('/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/test01.csv')

In [ ]:
online_sparse = pd.read_csv('/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_sparse.csv')

In [ ]:
cust_list=online_sparse[['Cust']]

In [ ]:

\

In [ ]:
online.drop(['금융/보험서비스','기타상품'], axis=1, inplace=True)

In [ ]:
#R = np.array(online_zero)
#R = R[:,2:]

In [ ]:
# Not higm dim
#R.shape

In [ ]:
online_zero = online.fillna(0)

In [ ]:
online_zero=online_zero.iloc[:,1:]

In [ ]:
online_zero['sum']=online_zero.sum(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [ ]:
online_sum=online_zero.sort_values(by=['sum'], ascending=False)

In [ ]:
def q4cut(s):
    return pd.qcut(s, 4, labels = ["G1","G2","G3","VIP"]).astype(str)

In [ ]:
online_sum['grade']=online_sum['sum'].transform(q4cut)

In [ ]:
#online_sum['sum'].quantile(q=0.75, interpolation='nearest')

In [ ]:
online_sum.shape

(26917, 61)

In [ ]:
# 최종 데이터 저장
online_sum.to_csv('/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_sum.csv', sep=',')

In [ ]:
grade_VIP=duckdb.query("select * from online_sum where grade='VIP'").to_df()

In [ ]:
grade_G3=duckdb.query("select * from online_sum where grade='G3'").to_df()

In [ ]:
grade_G2=duckdb.query("select * from online_sum where grade='G2'").to_df()

In [ ]:
grade_G1=duckdb.query("select * from online_sum where grade='G1'").to_df()

In [ ]:
grade_VIP=grade_VIP.drop(['Cust_2','grade','sum'], axis=1)

In [ ]:
grade_G3=grade_G3.drop(['Cust_2','grade','sum'], axis=1)
grade_G2=grade_G2.drop(['Cust_2','grade','sum'], axis=1)
grade_G1=grade_G1.drop(['Cust_2','grade','sum'], axis=1)

In [ ]:
# 단가 차이 있으니까 (가우시안) 표준화 or 정규화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler01 = MinMaxScaler()
scaler01.fit(grade_VIP)
scaler01_scaled = scaler01.transform(grade_VIP)

In [ ]:
scaler02 = MinMaxScaler()
scaler02.fit(grade_G3)
scaler02_scaled = scaler02.transform(grade_G3)

In [ ]:
scaler03 = MinMaxScaler()
scaler03.fit(grade_G2)
scaler03_scaled = scaler03.transform(grade_G2)

In [ ]:
scaler04 = MinMaxScaler()
scaler04.fit(grade_G1)
scaler04_scaled = scaler04.transform(grade_G1)

In [ ]:
R_VIP=scaler01_scaled
R_G3=scaler02_scaled
R_G2=scaler03_scaled
R_G1=scaler04_scaled

## Recommand System using Matrix Factorization

In [ ]:
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter

        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update

        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose

    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):

            # rating이 존재하는 index를 기준으로 training
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0:
                        self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2)
        return np.sqrt(cost) / len(xi)


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)


    def print_results(self):
        """
        print fit results
        """

        print("User Latent P:")
        print(self._P)
        print("Item Latent Q:")
        print(self._Q.T)
        print("P x Q:")
        print(self._P.dot(self._Q.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_P)
        print("Item Latent bias:")
        print(self._b_Q)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])

    def out(self):
      return self._P, self._Q.T, self.get_complete_matrix()    


## Fitting the model

In [ ]:
# run total data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_vip = MatrixFactorization(R_VIP, k=4, learning_rate=0.1, reg_param=0.1, epochs=10, verbose=True)
    factorizer_vip.fit()
    factorizer_vip.print_results()

    sol_vip=factorizer_vip.out()


In [ ]:
# 3.799^-5 user latent, item latent, final R
user_latent01=sol_vip[0]
item_latent01=sol_vip[1]
finalR01=sol_vip[2]

In [ ]:
R_G3=scaler02_scaled
R_G2=scaler03_scaled
R_G1=scaler04_scaled

In [ ]:
# run train data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5) R

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_G3 = MatrixFactorization(R_G3, k=4, learning_rate=0.01, reg_param=0.01, epochs=10, verbose=True)
    factorizer_G3.fit()
    factorizer_G3.print_results()

    sol_G3=factorizer_G3.out()

In [ ]:
# user latent, item latent, final R
user_latent02=sol_G3[0]
item_latent02=sol_G3[1]
finalR02=sol_G3[2]

In [ ]:
# run test data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5), R

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_G2 = MatrixFactorization(R_G2, k=4, learning_rate=0.01, reg_param=0.01, epochs=10, verbose=True)
    factorizer_G2.fit()
    factorizer_G2.print_results()

    sol_G2=factorizer_G2.out()

In [ ]:
# user latent, item latent, final R
user_latent03=sol_G2[0]
item_latent03=sol_G2[1]
finalR03=sol_G2[2]

In [ ]:
# run test data
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5), R

    # P, Q is (7 X k), (k X 5) matrix
    factorizer_G1 = MatrixFactorization(R_G1, k=4, learning_rate=0.1, reg_param=0.1, epochs=10, verbose=True)
    factorizer_G1.fit()
    factorizer_G1.print_results()

    sol_G1=factorizer_G1.out()

In [ ]:
# user latent, item latent, final R
user_latent04=sol_G1[0]
item_latent04=sol_G1[1]
finalR04=sol_G1[2]

## Save Results

In [ ]:
# inverse Minmax
finalR01_ori = scaler01.inverse_transform(finalR01)
finalR02_ori = scaler02.inverse_transform(finalR02)
finalR03_ori = scaler03.inverse_transform(finalR03)
finalR04_ori = scaler04.inverse_transform(finalR04)

In [ ]:
#np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_VIP.csv", finalR01, delimiter=",")

In [ ]:
#np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_G3.csv", finalR02, delimiter=",")

In [ ]:
#np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_G2.csv", finalR03, delimiter=",")

In [ ]:
#np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_G1.csv", finalR04, delimiter=",")

In [ ]:
#np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_VIP_ori.csv", finalR01_ori, delimiter=",")

In [ ]:
np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_G3_ori.csv", finalR02_ori, delimiter=",")

In [ ]:
np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_G2_ori.csv", finalR03_ori, delimiter=",")

In [ ]:
np.savetxt("/content/drive/Shareddrives/22SummerProjects/LPoint/Clustering/online_G1_ori.csv", finalR04_ori, delimiter=",")